TODO(Vlad): convert into a unit test.

# Imports

In [1]:
import pyarrow

import helpers.hparquet as hparque

/app/amp/helpers/hparquet.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
filters = [("currency_pair", "in", ["BTC_USDT"])]
file_name = "s3://cryptokaizen-data-test/v3/periodic_daily/airflow/downloaded_1min/parquet/ohlcv/futures/v7_3/ccxt/binance/v1_0_0/"

In [3]:
pyarrow.__version__

'14.0.2'

# Rationale

From the https://github.com/cryptokaizen/cmamp/issues/7097#issuecomment-1944181433

The notebook verifies that combining new data, saved and read using pyarrow version 14.0.0, with the existing production dataset doesn't impact performance when handling large parquet datasets and does not lead to any issues.

- Considering that we have the snippet of the preprod data at: "s3://cryptokaizen-data-test/v3/periodic_daily/airflow/downloaded_1min/parquet/ohlcv/futures/v7_3/ccxt/binance/v1_0_0/".
- Read the data from given location using `hparque.from_parquet()`
- Download the new portion of the data to the same location
- Read the mixed data

Expected behavior: the reading of large Parquet files does not increases the performance time and does not lead to any errors.

# Read initial data from preprod

In [22]:
df = hparque.from_parquet(file_name, filters=filters, aws_profile="ck")

In [23]:
df.head()

,timestamp,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
timestamp,,,,,,,,,,,
2019-10-01 00:00:00+00:00,1569888000000,8285.31,8312.30,8277.79,8283.14,225.421,binance,2022-07-09 12:07:51.240219+00:00,BTC_USDT,2019,10
2019-10-01 00:01:00+00:00,1569888060000,8279.98,8283.53,8268.33,8270.87,10.650,binance,2022-06-24 05:47:16.075108+00:00,BTC_USDT,2019,10
2019-10-01 00:02:00+00:00,1569888120000,8271.04,8271.40,8265.32,8265.32,16.916,binance,2022-07-09 12:07:51.240219+00:00,BTC_USDT,2019,10
2019-10-01 00:03:00+00:00,1569888180000,8265.32,8265.32,8240.53,8242.59,188.706,binance,2022-07-09 12:07:51.240219+00:00,BTC_USDT,2019,10
2019-10-01 00:04:00+00:00,1569888240000,8242.87,8249.18,8240.34,8244.03,36.597,binance,2022-07-09 12:07:51.240219+00:00,BTC_USDT,2019,10


In [24]:
df.shape

(2353346, 11)

# Download data

In [25]:
!/app/amp/im_v2/common/data/extract/download_bulk.py \
    --start_timestamp '2023-01-01T00:00:00+00:00' \
    --end_timestamp '2023-01-31T23:59:00+00:00' \
    --vendor 'ccxt' \
    --exchange_id 'binance' \
    --universe 'v7.3' \
    --data_type 'ohlcv' \
    --contract_type 'futures' \
    --aws_profile 'ck' \
    --assert_on_missing_data \
    --s3_path 's3://cryptokaizen-data-test' \
    --download_mode 'periodic_daily' \
    --downloading_entity 'airflow' \
    --action_tag 'downloaded_1min' \
    --data_format 'parquet'

INFO: > cmd='/app/amp/im_v2/common/data/extract/download_bulk.py --start_timestamp 2023-01-01T00:00:00+00:00 --end_timestamp 2023-01-31T23:59:00+00:00 --vendor ccxt --exchange_id binance --universe v7.3 --data_type ohlcv --contract_type futures --aws_profile ck --assert_on_missing_data --s3_path s3://cryptokaizen-data-test --download_mode periodic_daily --downloading_entity airflow --action_tag downloaded_1min --data_format parquet'
INFO: Saving log to file '/app/amp/im_v2/common/data/extract/download_bulk.py.log'
07:49:16 - INFO  dataset_schema_utils.py _get_dataset_schema_file_path:38 Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
07:49:16 - INFO  dataset_schema_utils.py get_dataset_schema:73          Loaded dataset schema version v3
07:49:16 - INFO  extractor.py __init__:42                               CCXT version: 4.2.13
07:49:20 - INFO  dataset_schema_utils.py _get_dataset_schema_file_path:38 Loading dataset schema file: /app/amp

100%|███████████████████████████████████████████| 45/45 [00:24<00:00,  1.81it/s]
07:55:35 - INFO  extract_utils.py process_downloaded_historical_data:942 Saving the dataset into s3://cryptokaizen-data-test/v3/periodic_daily/airflow/downloaded_1min/parquet/ohlcv/futures/v7_3/ccxt/binance/v1_0_0
07:55:36 - WARN  extract_utils.py verify_schema:1117                    Actual type=datetime64[us, UTC] != expected type=datetime64[ns, UTC] because of the different unit, continuing.
100%|███████████████████████████████████████████| 45/45 [00:24<00:00,  1.86it/s]
07:56:01 - INFO  extract_utils.py process_downloaded_historical_data:942 Saving the dataset into s3://cryptokaizen-data-test/v3/periodic_daily/airflow/downloaded_1min/parquet/ohlcv/futures/v7_3/ccxt/binance/v1_0_0
07:56:01 - WARN  extract_utils.py verify_schema:1117                    Actual type=datetime64[us, UTC] != expected type=datetime64[ns, UTC] because of the different unit, continuing.
100%|█████████████████████████████████████

# Read data after downloading

In [26]:
df = hparque.from_parquet(file_name, filters=filters, aws_profile="ck")

In [27]:
df.head()

,timestamp,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
timestamp,,,,,,,,,,,
2019-10-01 00:00:00+00:00,1569888000000,8285.31,8312.30,8277.79,8283.14,225.421,binance,2022-07-09 12:07:51.240219+00:00,BTC_USDT,2019,10
2019-10-01 00:01:00+00:00,1569888060000,8279.98,8283.53,8268.33,8270.87,10.650,binance,2022-06-24 05:47:16.075108+00:00,BTC_USDT,2019,10
2019-10-01 00:02:00+00:00,1569888120000,8271.04,8271.40,8265.32,8265.32,16.916,binance,2022-07-09 12:07:51.240219+00:00,BTC_USDT,2019,10
2019-10-01 00:03:00+00:00,1569888180000,8265.32,8265.32,8240.53,8242.59,188.706,binance,2022-07-09 12:07:51.240219+00:00,BTC_USDT,2019,10
2019-10-01 00:04:00+00:00,1569888240000,8242.87,8249.18,8240.34,8244.03,36.597,binance,2022-07-09 12:07:51.240219+00:00,BTC_USDT,2019,10


In [28]:
df.shape

(2397986, 11)